In [9]:
#library
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pandas as pd
import gensim
import regex as re
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim_models
from pprint import pprint
import numpy as np

import pyLDAvis.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [10]:
#read data
data = pd.read_csv('hasil.csv')
data = data.drop(columns=['Companies','Locations'])

#formating text lowercasing remove symbol
stop_words = set(stopwords.words('english'))
data['Descriptions_without_stopwords'] = data['Descriptions'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data['Descriptions_without_stopwords'] = data['Descriptions_without_stopwords'].str.replace('[^\w\s]', '')

# data['Descriptions_without_stopwords'] = data['Descriptions_without_stopwords'].map(re.sub(r"(\w)([A-Z])", r"\1 \2", ele) for ele in test_list)
data['Descriptions_without_stopwords'] = data['Descriptions_without_stopwords'].map(lambda x: x.lower())

<>:8: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \w
C:\Users\helmi\AppData\Local\Temp\ipykernel_5644\2655294412.py:8: DeprecationWarning: invalid escape sequence \w
  data['Descriptions_without_stopwords'] = data['Descriptions_without_stopwords'].str.replace('[^\w\s]', '')
C:\Users\helmi\AppData\Local\Temp\ipykernel_5644\2655294412.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Descriptions_without_stopwords'] = data['Descriptions_without_stopwords'].str.replace('[^\w\s]', '')


In [11]:
def sent_to_words (sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
        if word not in stop_words] for doc in texts]

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(w) for w in text.split(' ')])

In [12]:
data['Descriptions_without_stopwords'] = data['Descriptions_without_stopwords'].apply(lemmatize_text)
dataLDA = data.Descriptions_without_stopwords.tolist()
data_words = list(sent_to_words(dataLDA))
data_words = remove_stopwords(data_words)
print(data_words)

#id2text
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]

[['job', 'computer', 'science', 'information', 'system', 'relevant', 'fieldprior', 'experience', 'system', 'analyst', 'similar', 'qualification', 'plusstrong', 'analytical', 'problemsolving', 'skillsexcellent', 'communication', 'skillsability', 'work', 'well', 'teammicrosoft', 'officesqlhaving', 'skill', 'programming', 'language', 'php', 'javascript', 'framework', 'experience', 'application', 'working', 'visit', 'office', 'twice', 'one', 'week'], ['job', 'job', 'sourced', 'job', 'board', 'learn', 'deploy', 'maintain', 'troubleshoot', 'core', 'business', 'application', 'including', 'application', 'server', 'associated', 'hardware', 'endpoint', 'databasesmeet', 'coordinate', 'internal', 'external', 'stakeholder', 'establish', 'project', 'scope', 'system', 'goal', 'analyze', 'prioritize', 'organize', 'requirement', 'specification', 'data', 'mapping', 'diagram', 'flowchart', 'developer', 'tester', 'followtranslate', 'highly', 'technical', 'specification', 'clear', 'nontechnical', 'setup', 

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = id2word, num_topics=30, update_every=1, chunksize=100, passes=10, alpha="auto")


In [14]:
result = lda_model.print_topics(num_words=3, num_topics=3)
print(result)

[(18, '0.003*"basic" + 0.003*"codehave" + 0.003*"backgroundfresh"'), (10, '0.045*"experience" + 0.045*"system" + 0.023*"communication"'), (9, '0.048*"system" + 0.024*"data" + 0.019*"job"')]


In [15]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(lda_model, corpus, data_words)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,10.0,0.9783,"experience, system, communication, analyst, ph...","[job, computer, science, information, system, ..."
1,1,9.0,0.9957,"system, data, job, maintenance, information, b...","[job, job, sourced, job, board, learn, deploy,..."
2,2,1.0,0.9526,"reporting, sql, banking, information, job, bir...","[job, job, sourced, job, board, learn, ops, re..."
3,3,6.0,0.9883,"engineering, job, interest, logic, development...","[job, job, sourced, job, board, learn, dosoft,..."
4,4,9.0,0.9950,"system, data, job, maintenance, information, b...","[job, company, description, create, digital, p..."
5,5,9.0,0.9874,"system, data, job, maintenance, information, b...","[job, job, sourced, job, board, learn, perform..."
6,6,9.0,0.9961,"system, data, job, maintenance, information, b...","[job, maintain, troubleshoot, core, business, ..."


In [24]:
print(id2word[9])

javascript


In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word,mds='mmds',R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.105464 -0.139715       1        1  69.614666
1      0.232261  0.150979       2        1  14.374504
6     -0.204286  0.188256       3        1  10.150332
10    -0.205407 -0.168651       4        1   5.334674
23     0.003045 -0.001789       5        1   0.023820
15     0.002867 -0.001592       6        1   0.022351
13     0.002800 -0.001422       7        1   0.022303
5      0.002523 -0.001314       8        1   0.022234
8      0.003682 -0.000283       9        1   0.021904
26     0.002849 -0.001641      10        1   0.020630
22     0.002394 -0.001364      11        1   0.020568
7      0.002771 -0.001228      12        1   0.020550
2      0.002813 -0.001048      13        1   0.020539
14     0.002758 -0.001204      14        1   0.020537
21     0.002592 -0.001024      15        1   0.020532
17     0.002733 -0.001194      16        1   0.020531
20     0.002737 -0.001190      17        1   0.020528
11     0.002728 -0.001160      18        1   0.020503
12     0.002752 -0.001123      19        1   0.020495
4      0.002753 -0.001093      20        1   0.020471
24     0.002720 -0.001130      21        1   0.020420
27     0.002717 -0.001119      22        1   0.018545
28     0.002717 -0.001119      23        1   0.018545
25     0.002717 -0.001119      24        1   0.018545
0      0.002717 -0.001119      25        1   0.018545
19     0.002717 -0.001119      26        1   0.018545
18     0.002717 -0.001119      27        1   0.018545
16     0.002717 -0.001119      28        1   0.018545
3      0.002717 -0.001119      29        1   0.018545
29     0.002717 -0.001119      30        1   0.018545, topic_info=           Term       Freq      Total Category  logprob  loglift
26       system  24.000000  24.000000  Default   30.000  30.0000
50         data  12.000000  12.000000  Default   29.000  29.0000
85  maintenance   8.000000   8.000000  Default   28.000  28.0000
10          job  15.000000  15.000000  Default   27.000  27.0000
43     business   6.000000   6.000000  Default   26.000  26.0000
..          ...        ...        ...      ...      ...      ...
18  programming   0.000428   2.851694  Topic30   -5.663  -0.2126
20     relevant   0.000428   3.720260  Topic30   -5.663  -0.4785
21      science   0.000428   2.851823  Topic30   -5.663  -0.2126
23        skill   0.000428   3.665812  Topic30   -5.663  -0.4637
26       system   0.000428  24.949175  Topic30   -5.663  -2.3815

[1805 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
34        1  0.962844        ability
273       1  0.905302          agile
227       1  0.906557          alarm
35        1  0.963185  alarmsperform
189       1  0.506440      algorithm
...     ...       ...            ...
33        2  0.353336        working
33        4  0.176668        working
134       1  0.619932           year
134       2  0.206644           year
134       3  0.206644           year

[340 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 7, 11, 24, 16, 14, 6, 9, 27, 23, 8, 3, 15, 22, 18, 21, 12, 13, 5, 25, 28, 29, 26, 1, 20, 19, 17, 4, 30])